In [1]:
#구글 드라이브와 mount(연결)
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [3]:
!pip install pyproj
!pip install haversine
!pip install dbfread
!pip install gurobipy
!pip install shapely
!pip install geopandas

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 64.8 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.7/12.7 MB 59.4 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 52.4 MB/s eta 0:00:00


In [4]:
import pandas as pd
import numpy as np
import pyproj
# 지도 시각화
import folium
from folium import Marker
from folium.plugins import MiniMap
from folium.plugins import MarkerCluster
import warnings
warnings.filterwarnings('ignore')
from haversine import haversine, Unit

import seaborn as sns
from scipy.spatial import distance_matrix
from gurobipy import *
from scipy.spatial import ConvexHull
from shapely.geometry import Polygon, Point
from numpy import random
import time
from shapely import *
import requests
import json
import geopandas as gpd
import matplotlib.pyplot as plt
import matplotlib as mpl
import platform
from matplotlib import font_manager, rc
from shapely.geometry.point import Point
import re

In [ ]:
# 파이썬 시각화 패키지 불러오기
#import matplotlib.pyplot as plt
%matplotlib inline

# 사용자 운영체제 확인
import platform
platform.system()

# 운영체제별 한글 폰트 설정
if platform.system() == 'Darwin': # Mac 환경 폰트 설정
    plt.rc('font', family='AppleGothic')

plt.rc('axes', unicode_minus=False) # 마이너스 폰트 설정

# 글씨 선명하게 출력하는 설정
%config InlineBackend.figure_format = 'retina'

In [ ]:
# 서울시 전동킥보드 견인 현황
es_tow = pd.read_csv('/content/drive/MyDrive/캡스톤디자인/data/서울시 전동킥보드 견인 현황.csv', encoding = 'euc-kr')

In [ ]:
es_tow.head()

,번호,신고일시,구정보,주소,유형,조치일
0,9999,2021-10-15,송파구,서울특별시 송파구 오금동 89-7,"점자블록, 엘리베이터 입구",2021-10-15
1,9998,2021-10-15,송파구,서울특별시 송파구 장지동 850,"점자블록, 엘리베이터 입구",2021-10-15
2,9997,2021-10-15,동작구,서울특별시 동작구 사당동 583-11,"점자블록, 엘리베이터 입구",2021-10-15
3,9996,2021-10-15,관악구,서울특별시 관악구 봉천동 979-1,지하철역 진출입로 및 주변,2021-10-15
4,9995,2021-10-15,관악구,서울특별시 관악구 봉천동 1693-39,"횡단보도, 산책로 등",2021-10-15


In [ ]:
es_tow.tail()

,번호,신고일시,구정보,주소,유형,조치일
60600,10000,2021-10-15,관악구,서울특별시 관악구 봉천동 1685-42,"버스정류장, 택시 승강장",2021-10-15
60601,1000,2021-07-27,마포구,서울특별시 마포구 망원동 452-1,보도와 차도가 구분된 도로의 차도,2021-07-27
60602,100,2021-07-16,마포구,서울특별시 마포구 서교동 446-16,보도와 차도가 구분된 도로의 차도,2021-07-16
60603,10,2021-07-15,동작구,서울특별시 동작구 상도동 159-275,기타,2021-07-20
60604,1,2021-07-15,동작구,서울특별시 동작구 상도동 215-45,"점자블록, 엘리베이터 입구",2021-07-15


In [ ]:
# 크기 확인하기
es_tow.shape

(60605, 6)

In [ ]:
# 정보 확인하기
es_tow.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60605 entries, 0 to 60604
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   번호      60605 non-null  int64 
 1   신고일시    60605 non-null  object
 2   구정보     60605 non-null  object
 3   주소      60605 non-null  object
 4   유형      60604 non-null  object
 5   조치일     60604 non-null  object
dtypes: int64(1), object(5)
memory usage: 2.8+ MB


In [ ]:
# 결측치 확인하기
es_tow.isnull().sum()

번호      0
신고일시    0
구정보     0
주소      0
유형      1
조치일     1
dtype: int64

In [ ]:
# 견인 유형별
es_tow_1 = es_tow[es_tow['유형'].str.contains('버스정류장, 택시 승강장')]
es_tow_1

ValueError: ignored

In [ ]:
# 서울시 전동킥보드 주차구역 현황
es_park = pd.read_csv('/content/drive/MyDrive/캡스톤디자인/data/서울시 전동킥보드 주차구역 현황.csv', encoding = 'euc-kr')
es_park = es_park.drop(['거치대 크기'], axis=1)

In [ ]:
es_park.head()

,순번,시군구명,주소,상세위치,거치대 유무
0,1,종로구,팔판동 115-63,청와대 춘추문 맞은편 인근,Y
1,2,종로구,연건동 218-1,KT광화문 혜화지사 앞,Y
2,3,종로구,연건동 178-3,홍익대학교 대학로 맞은편,Y
3,4,종로구,동승동 1-24,대학로 마로니에공원 앞,Y
4,5,종로구,와룡동 75-4,연악사 맞은편,Y


In [ ]:
es_park.shape

(193, 6)

In [ ]:
es_park.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 193 entries, 0 to 192
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   순번      193 non-null    int64  
 1   시군구명    193 non-null    object 
 2   주소      193 non-null    object 
 3   상세위치    193 non-null    object 
 4   거치대 유무  193 non-null    object 
 5   거치대 크기  82 non-null     float64
dtypes: float64(1), int64(1), object(4)
memory usage: 9.2+ KB


In [ ]:
es_park.isnull().sum()

순번          0
시군구명        0
주소          0
상세위치        0
거치대 유무      0
거치대 크기    111
dtype: int64

In [ ]:
# 전동킥보드 거치대 Y 주차 현황
es_park_y = es_park[es_park['거치대 유무'].str.contains('Y')]
# 전동킥보드 거치대 N 주차 현황
es_park_n = es_park[es_park['거치대 유무'].str.contains('N')]

In [ ]:
es_park_y.head()

,순번,시군구명,주소,상세위치,거치대 유무
0,1,종로구,팔판동 115-63,청와대 춘추문 맞은편 인근,Y
1,2,종로구,연건동 218-1,KT광화문 혜화지사 앞,Y
2,3,종로구,연건동 178-3,홍익대학교 대학로 맞은편,Y
3,4,종로구,동승동 1-24,대학로 마로니에공원 앞,Y
4,5,종로구,와룡동 75-4,연악사 맞은편,Y


In [ ]:
es_park_n.head()

,순번,시군구명,주소,상세위치,거치대 유무
21,22,용산구,용산구 갈월동 69-29,숙대입구역 10번 출구 앞,N
26,27,성동구,홍익동 614-1,상왕십리역 1번 출구 따릉이 거치대 인근,N
27,28,광진구,자양동854,뚝섬유원지역 1번 축구 인근,N
28,29,동대문구,장안동 416-10,창도빌딩 앞(지하철 환풍구 옆 가로등 가로수 사이),N
29,30,동대문구,답십리동 497-23,천호대로 257 청계푸르지오시티오피스텔 앞,N


In [ ]:
# 서울시 따릉이 대여소 마스터 정보
bc_rent = pd.read_csv('/content/drive/MyDrive/캡스톤디자인/data/서울시 따릉이대여소 마스터 정보.csv', encoding = 'euc-kr')
bc_rent.head()

,대여소_ID,주소1,주소2,위도,경도
0,ST-999,서울특별시 양천구 목동서로 280,목동아파트 8단지 상가동,0.000000,0.000000
1,ST-998,서울특별시 양천구 목동서로 130,목동아파트 4단지 상가동,0.000000,0.000000
2,ST-997,서울특별시 양천구 목동중앙로 49,목동3단지 시내버스정류장,37.534390,126.869598
3,ST-996,서울특별시 양천구 남부순환로88길5-16,양강중학교앞 교차로,37.524334,126.850548
4,ST-995,서울특별시 양천구 신정동 621-14,계남공원 입구 주출입구 좌측,37.510681,126.857399


In [ ]:
bc_rent.shape

(3208, 5)

In [ ]:
bc_rent.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3208 entries, 0 to 3207
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   대여소_ID  3208 non-null   object 
 1   주소1     3208 non-null   object 
 2   주소2     1543 non-null   object 
 3   위도      3208 non-null   float64
 4   경도      3208 non-null   float64
dtypes: float64(2), object(3)
memory usage: 125.4+ KB


In [ ]:
bc_rent.isnull().sum()

대여소_ID       0
주소1          0
주소2       1665
위도           0
경도           0
dtype: int64

In [ ]:
# FutureWarning 제거
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [ ]:
# 서울시 중심부의 위도, 경도
center = [37.541, 126.986]

# 맵이 center 에 위치하고, zoom 레벨은 11로 시작하는 맵 m을 만듬
m = folium.Map(location=center, zoom_start=10)

In [ ]:
# 서울특별시 송파구 데이터만 추출
bc_rent_sp = bc_rent[bc_rent['주소1'].str.contains('서울특별시 송파구')]
bc_rent_sp.head()

,대여소_ID,주소1,주소2,위도,경도
19,ST-981,서울특별시 송파구 중대로 16,송파구 중대로 16,37.489872,127.110840
72,ST-933,서울특별시 송파구 백제고분로 188,잠실포스코더샵,37.504200,127.087532
75,ST-930,서울특별시 송파구 오금로 지하 499,3번출구,37.493343,127.144730
99,ST-909,서울특별시 송파구 도곡로 446,송파구 잠실동 128,37.505463,127.077347
100,ST-908,서울특별시 송파구 위례성대로 114,송파구 방이동 75,37.513962,127.121674


In [ ]:
bc_rent_splat = [x for x in bc_rent_sp['위도']]
bc_rent_splon = [x for x in bc_rent_sp['경도']]

In [ ]:
for lat, lon in zip(bc_rent_splat, bc_rent_splon):
  folium.Marker(location = [lat,lon],
           icon=folium.Icon(color='green',icon='flag')
          ).add_to(m)
m

In [ ]:
# 서울시 따릉이 대여소별 대여/반납 승객수 정보
bc_rent_count = pd.read_csv('/content/drive/MyDrive/캡스톤디자인/data/tpss_bcycl_od_statnhm_20230322 2.csv', encoding = 'euc-kr')
bc_rent_count.head()

,기준_날짜,집계_기준,기준_시간대,시작_대여소_ID,시작_대여소명,종료_대여소_ID,종료_대여소명,전체_건수,전체_이용_분,전체_이용_거리
0,20230322,출발시간,0,ST-1184,수서동_025_1,ST-1184,수서동_025_1,1,0,0
1,20230322,출발시간,0,ST-1376,남영동_003_1,ST-1376,남영동_003_1,1,0,0
2,20230322,출발시간,0,ST-1751,수색동_031_1,ST-1751,수색동_031_1,1,0,0
3,20230322,출발시간,0,ST-2678,대치2동_062_1,ST-2678,대치2동_062_1,1,0,0
4,20230322,출발시간,0,ST-536,화곡1동_060_1,ST-536,화곡1동_060_1,1,0,0


In [ ]:
bc_rent_count.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 299583 entries, 0 to 299582
Data columns (total 10 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   기준_날짜      299583 non-null  int64 
 1   집계_기준      299583 non-null  object
 2   기준_시간대     299583 non-null  int64 
 3   시작_대여소_ID  299583 non-null  object
 4   시작_대여소명    299583 non-null  object
 5   종료_대여소_ID  299583 non-null  object
 6   종료_대여소명    298368 non-null  object
 7   전체_건수      299583 non-null  int64 
 8   전체_이용_분    299583 non-null  int64 
 9   전체_이용_거리   299583 non-null  int64 
dtypes: int64(5), object(5)
memory usage: 22.9+ MB


In [ ]:
bc_rent_count.isnull().sum()

기준_날짜           0
집계_기준           0
기준_시간대          0
시작_대여소_ID       0
시작_대여소명         0
종료_대여소_ID       0
종료_대여소명      1215
전체_건수           0
전체_이용_분         0
전체_이용_거리        0
dtype: int64

In [ ]:
# 지하철역(1호선~8호선) 위/경도
subway = pd.read_csv('/content/drive/MyDrive/캡스톤디자인/data/서울교통공사_1_8호선 역사 좌표(위경도) 정보_20211231.csv', encoding = 'euc-kr')
subway = subway.drop(['작성일자'], axis=1)    # 열 삭제
subway.info()
subway.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 276 entries, 0 to 275
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   연번            276 non-null    int64  
 1   호선            276 non-null    int64  
 2   고유역번호(외부역코드)  276 non-null    int64  
 3   역명            276 non-null    object 
 4   위도            276 non-null    float64
 5   경도            276 non-null    float64
dtypes: float64(2), int64(3), object(1)
memory usage: 13.1+ KB


,연번,호선,고유역번호(외부역코드),역명,위도,경도
0,1,1,150,서울,37.553150,126.972533
1,2,1,151,시청,37.563590,126.975407
2,3,1,152,종각,37.570203,126.983116
3,4,1,153,종로3가,37.570429,126.992095
4,5,1,154,종로5가,37.570971,127.001900


In [ ]:
subway.tail()

,연번,호선,고유역번호(외부역코드),역명,위도,경도
271,272,8,2823,남한산성입구,37.451568,127.159845
272,273,8,2824,단대오거리,37.445057,127.156735
273,274,8,2825,신흥,37.440952,127.147590
274,275,8,2826,수진,37.437575,127.140936
275,276,8,2827,모란,37.433888,127.129921


In [ ]:
subway_lat = [x for x in subway['위도']]
subway_lon = [x for x in subway['경도']]

In [ ]:
# m1은 지도에 지하철역 시각화
m1 = folium.Map(location=center, zoom_start=10)

for lat, lon in zip(subway_lat, subway_lon):
  folium.Marker(location = [lat,lon],
           icon=folium.Icon(color='purple',icon='car')
          ).add_to(m1)
m1

In [ ]:
# 5호선 8호선 데이터만 추출
subway_5= subway[subway['호선'] == 5]
subway_8= subway[subway['호선'] == 8]
subway_58 = pd.concat([subway_5, subway_8], axis = 0)
subway_58

,연번,호선,고유역번호(외부역코드),역명,위도,경도
121,122,5,2511,방화,37.577669,126.812822
122,123,5,2512,개화산,37.572458,126.806838
123,124,5,2513,김포공항,37.562170,126.801273
124,125,5,2514,송정,37.561411,126.812052
125,126,5,2515,마곡,37.562182,126.826930
...,...,...,...,...,...,...
271,272,8,2823,남한산성입구,37.451568,127.159845
272,273,8,2824,단대오거리,37.445057,127.156735
273,274,8,2825,신흥,37.440952,127.147590
274,275,8,2826,수진,37.437575,127.140936


In [ ]:
# m2은 지도에 지하철역(5, 8호선) + 송파구 따릉이 시각화
m2 = folium.Map(location=center, zoom_start=10)

subway_58_lat = [x for x in subway['위도']]
subway_58_lon = [x for x in subway['경도']]

# 보라색 : 지하철역
for lat, lon in zip(subway_58_lat, subway_58_lon):
  folium.Marker(location = [lat,lon],
           icon=folium.Icon(color='purple',icon='flag')
          ).add_to(m2)

# 녹색 : 따릉이
for lat, lon in zip(bc_rent_splat, bc_rent_splon):
  folium.Marker(location = [lat,lon],
           icon=folium.Icon(color='green',icon='flag')
          ).add_to(m2)

m2

In [ ]:
# 서울시 버스 정류소 위치 데이터 불러오기
busstop = pd.read_excel('/content/drive/MyDrive/캡스톤디자인/data/서울시 버스정류소 좌표데이터(2023.03.10).xlsx')
busstop.head()

,NODE_ID,ARS_ID,정류소명,X좌표,Y좌표
0,100000001,1001,종로2가사거리,126.987750,37.569765
1,100000002,1002,창경궁.서울대학교병원,126.996566,37.579183
2,100000003,1003,명륜3가.성대입구,126.998340,37.582671
3,100000004,1004,종로2가.삼일교,126.987613,37.568579
4,100000005,1005,혜화동로터리.여운형활동터,127.001744,37.586243


In [7]:
people = pd.read_csv('/content/drive/MyDrive/캡스톤디자인/data/행정동별 거주인구.csv', encoding = 'euc-kr')
people


,기준년월(STD_YM),행정동코드(ADMI_CD),행정동명(ADMI_NM),성별코드(SEXDSTN_CD),연령대코드(AGRDE_CD),거주인구_수(RSPOP_CNT),시군구명(CTY_NM)
0,202005,11350640,번1동,1,1014,24,금천구
1,201204,11230730,용산2가동,1,7074,17,광진구
2,202105,11260690,등촌3동,2,3034,674,동대문구
3,201209,11500591,대림1동,2,5559,103,광진구
4,202106,11350560,삼청동,2,8084,1456,송파구
...,...,...,...,...,...,...,...
495,201204,11215840,우장산동,2,6064,751,송파구
496,201210,11500535,충현동,1,6569,5,마포구
497,202004,11545610,답십리2동,2,I00,8,성동구
498,201803,11410555,암사1동,2,5054,804,동작구


In [11]:
# Convert '기준년월(STD_YM)' column to datetime type
people['기준년월(STD_YM)'] = pd.to_datetime(people['기준년월(STD_YM)'], format='%Y%m')

people2022 = people[people['기준년월(STD_YM)'].dt.year == 2021]
people2022

,기준년월(STD_YM),행정동코드(ADMI_CD),행정동명(ADMI_NM),성별코드(SEXDSTN_CD),연령대코드(AGRDE_CD),거주인구_수(RSPOP_CNT),시군구명(CTY_NM)
2,2021-05-01,11260690,등촌3동,2,3034,674,동대문구
4,2021-06-01,11350560,삼청동,2,8084,1456,송파구
35,2021-02-01,11440630,방화1동,2,9599,80,종로구
47,2021-04-01,11740640,잠실6동,2,8084,374,종로구
51,2021-01-01,11380690,신정7동,2,8589,1312,성동구
52,2021-01-01,11680655,세곡동,1,8084,488,강북구
54,2021-01-01,11710520,보광동,1,8084,510,광진구
72,2021-06-01,11110570,성산2동,2,2529,557,중랑구
87,2021-06-01,11680640,여의동,1,2024,1283,강북구
110,2021-05-01,11470570,독산3동,2,8084,213,영등포구


In [12]:
people마포구 = people[people['시군구명(CTY_NM)'] == '마포구']
people마포구

,기준년월(STD_YM),행정동코드(ADMI_CD),행정동명(ADMI_NM),성별코드(SEXDSTN_CD),연령대코드(AGRDE_CD),거주인구_수(RSPOP_CNT),시군구명(CTY_NM)
8,2014-06-01,11170690,쌍문2동,1,9999,515,마포구
18,2017-06-01,11650620,월계3동,1,1519,1118,마포구
28,2017-07-01,11290810,동화동,2,4549,385,마포구
108,2016-10-01,11710647,남가좌2동,2,6064,745,마포구
116,2020-05-01,11470590,전농2동,2,5054,8,마포구
120,2017-05-01,11545690,가락1동,1,4549,642,마포구
125,2013-03-01,11215760,창3동,2,7579,1207,마포구
156,2018-01-01,11710631,신정6동,1,8589,4,마포구
191,2013-01-01,11590670,번1동,2,9094,26,마포구
222,2017-03-01,11320690,쌍문3동,2,0509,33,마포구


In [13]:
people2022 = pd.read_csv('/content/drive/MyDrive/캡스톤디자인/data/LOCAL_PEOPLE_GU_2022.csv', encoding = 'euc-kr')
people2022

,기준일ID,시간대구분,자치구코드,총생활인구수,남자0세부터9세생활인구수,남자10세부터14세생활인구수,남자15세부터19세생활인구수,남자20세부터24세생활인구수,남자25세부터29세생활인구수,남자30세부터34세생활인구수,...,여자25세부터29세생활인구수,여자30세부터34세생활인구수,여자35세부터39세생활인구수,여자40세부터44세생활인구수,여자45세부터49세생활인구수,여자50세부터54세생활인구수,여자55세부터59세생활인구수,여자60세부터64세생활인구수,여자65세부터69세생활인구수,여자70세이상생활인구수
0,20220101,0,11110,198013.1166,4873.9724,2875.5716,4721.0371,8056.7447,8409.5262,7030.9611,...,8358.8772,6598.3393,7714.6240,6930.9907,8622.9096,7542.4183,8012.8912,6618.1705,5234.3484,14670.1388
1,20220101,0,11140,163439.6365,4588.8045,1754.1373,2565.4546,5964.4053,7538.1455,7272.5991,...,8907.1659,7146.4494,7837.6613,6241.1243,6190.8188,5122.5866,5497.6735,4887.9068,3580.1862,9786.9251
2,20220101,0,11170,252165.9659,7338.0378,3621.6062,5679.3041,8010.4273,10246.1419,11344.5582,...,11230.4114,11730.1838,13262.1450,10157.6281,10607.3606,9141.2146,8998.7112,8093.6815,5922.3439,15970.6930
3,20220101,0,11200,312343.4234,11468.7636,4805.9768,6325.7793,10097.0347,12308.5453,12072.8752,...,12258.5309,12985.5190,15908.0426,12635.4614,12772.6410,11329.8455,12209.6659,10498.3080,7920.3737,20142.3144
4,20220101,0,11215,348524.3239,10212.5621,5475.5993,8713.1763,12957.8151,16604.5445,14338.6849,...,17011.8494,14646.3396,15249.4805,13215.0753,14246.1921,12944.7686,14256.4084,11730.0618,8646.9569,19500.4808
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
218995,20221231,23,11620,468533.2179,11551.3074,5620.8956,9207.5815,18665.9541,30952.8003,25695.9631,...,27889.7343,20669.3039,19181.4143,15548.3493,16195.5139,14859.1897,15884.1065,13996.0120,11699.3191,29012.6524
218996,20221231,23,11650,464794.4332,20247.1960,10900.8345,16271.3110,14626.4441,15461.1142,15497.7090,...,15823.3834,15603.4387,20564.2567,20733.5833,23047.8588,16474.4033,15469.5319,12576.3360,10478.7079,26054.0498
218997,20221231,23,11680,626217.7021,24272.7828,15790.4802,23289.9467,18699.7994,22204.9649,22483.0248,...,24268.5446,22765.5867,28073.9601,28227.5084,31894.4841,21793.2518,19696.3223,15849.6223,13672.9699,34056.4408
218998,20221231,23,11710,726058.1872,31134.3399,15478.9444,20867.6177,20607.2146,25532.9245,26891.8841,...,26526.0862,28859.3884,35077.4438,31098.1011,31050.9682,26653.2291,28017.9861,23899.3768,19144.9527,41267.7877


In [17]:
people2022마포구 = people2022[people2022['자치구코드'] == '140']
people2022마포구

,기준일ID,시간대구분,자치구코드,총생활인구수,남자0세부터9세생활인구수,남자10세부터14세생활인구수,남자15세부터19세생활인구수,남자20세부터24세생활인구수,남자25세부터29세생활인구수,남자30세부터34세생활인구수,...,여자25세부터29세생활인구수,여자30세부터34세생활인구수,여자35세부터39세생활인구수,여자40세부터44세생활인구수,여자45세부터49세생활인구수,여자50세부터54세생활인구수,여자55세부터59세생활인구수,여자60세부터64세생활인구수,여자65세부터69세생활인구수,여자70세이상생활인구수


In [14]:
people2021 = pd.read_csv('/content/drive/MyDrive/캡스톤디자인/data/LOCAL_PEOPLE_GU_2021.csv', encoding = 'euc-kr')
people2021

,기준일ID,시간대구분,자치구코드,총생활인구수,남자0세부터9세생활인구수,남자10세부터14세생활인구수,남자15세부터19세생활인구수,남자20세부터24세생활인구수,남자25세부터29세생활인구수,남자30세부터34세생활인구수,...,여자25세부터29세생활인구수,여자30세부터34세생활인구수,여자35세부터39세생활인구수,여자40세부터44세생활인구수,여자45세부터49세생활인구수,여자50세부터54세생활인구수,여자55세부터59세생활인구수,여자60세부터64세생활인구수,여자65세부터69세생활인구수,여자70세이상생활인구수
0,20210101,0,11110,194924.6184,4920.3837,2992.2011,4479.0239,7799.4338,8270.0476,6591.1392,...,7513.2691,6532.6210,7281.9873,7159.4938,8479.3144,7389.4164,8051.4425,6722.6436,5109.8562,15036.7700
1,20210101,0,11140,148826.8808,3354.1662,1376.5528,2368.3576,4868.7237,6945.7261,6640.6784,...,7110.3397,6713.5499,7244.8277,5420.8522,5795.2854,4829.2206,5617.3479,4846.5084,3847.0501,9823.1857
2,20210101,0,11170,250141.2136,7083.5939,3542.5572,5294.7572,7602.9197,9970.4226,11424.4500,...,10681.6414,11129.4982,12831.7555,10190.8974,10715.4628,8839.3659,9417.1985,8353.8373,5899.2935,16888.6424
3,20210101,0,11200,315490.0745,11513.7075,4873.2608,6427.3620,9447.8712,12296.0848,12453.8504,...,12707.7943,13462.4842,15997.2596,12525.5609,12903.9998,10991.7768,12442.7269,10785.3456,7925.9533,20678.4090
4,20210101,0,11215,360268.0149,10668.3623,5764.4680,9216.3402,12610.7515,16765.3160,15174.9819,...,17406.2677,15460.2633,15679.1837,13760.7719,14761.8863,13689.3575,14573.9438,12408.8341,8957.7143,20651.2597
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
218995,20211231,23,11620,472514.0761,11459.7460,5587.5167,9246.5077,18907.9663,31630.4011,25874.5206,...,27657.9259,20556.6298,19093.2494,15704.3718,16656.3598,15146.7999,16492.3015,14879.3203,12008.2245,29786.2623
218996,20211231,23,11650,455062.9561,19787.1947,10682.4784,13954.8801,12218.0882,13859.9908,15248.2675,...,14424.0264,16067.1612,20992.0304,20998.7721,22433.3375,16798.5863,15450.9244,13399.7482,11023.4122,26408.4854
218997,20211231,23,11680,626466.4158,24454.1035,15836.2983,21877.8836,17167.4107,21281.2474,22822.7317,...,23086.3151,23453.2183,28196.1925,28974.8318,31343.5098,21532.9109,19701.3446,17174.8688,14571.0011,34306.4306
218998,20211231,23,11710,712411.1104,30428.8846,15366.8198,20377.0469,18965.2072,24192.2124,26429.5933,...,25489.0412,27829.3396,34237.0376,30491.9252,30678.1327,26101.4104,28698.9175,24997.5744,19119.4128,40250.2444
